# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [3]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [5]:
from src.data import data_collection

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [5]:
data_collection.download_data_and_load_into_sql()

DuplicateDatabase: database "opportunity_youth" already exists


Now it's time to explore the data!

In [6]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import matplotlib.pyplot as plt
%matplotlib inline

DBNAME = "opportunity_youth"

conn = psycopg2.connect(dbname=DBNAME)
cur = conn.cursor()

***
# Query database and creating DataFrame
#### df_pums_2017

In [7]:
#Querying SQL database

pd.read_sql("""SELECT *
            FROM pums_2017
            LIMIT 10;""", conn)

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000000006,9,01,11606,4,53,1061971,27.0,68.0,...,53.0,24.0,39.0,24.0,7.0,27.0,8.0,46.0,25.0,50.0
1,P,2013000000006,9,02,11606,4,53,1061971,22.0,66.0,...,49.0,21.0,38.0,20.0,7.0,25.0,8.0,41.0,22.0,47.0
2,P,2013000000012,9,01,10100,4,53,1061971,22.0,72.0,...,24.0,22.0,25.0,7.0,21.0,35.0,6.0,22.0,6.0,37.0
3,P,2013000000012,9,02,10100,4,53,1061971,19.0,64.0,...,21.0,18.0,19.0,7.0,17.0,29.0,6.0,19.0,6.0,29.0
4,P,2013000000038,9,01,11505,4,53,1061971,4.0,52.0,...,4.0,1.0,2.0,8.0,8.0,1.0,4.0,6.0,1.0,4.0
5,P,2013000000038,9,02,11505,4,53,1061971,4.0,51.0,...,4.0,1.0,1.0,8.0,7.0,1.0,4.0,7.0,2.0,4.0
6,P,2013000000038,9,03,11505,4,53,1061971,7.0,18.0,...,5.0,2.0,3.0,14.0,12.0,2.0,8.0,13.0,3.0,6.0
7,P,2013000000070,9,01,10400,4,53,1061971,15.0,59.0,...,26.0,14.0,14.0,26.0,15.0,16.0,33.0,16.0,15.0,15.0
8,P,2013000000070,9,02,10400,4,53,1061971,18.0,56.0,...,27.0,15.0,16.0,30.0,14.0,18.0,38.0,17.0,17.0,14.0
9,P,2013000000082,9,01,11615,4,53,1061971,90.0,40.0,...,119.0,152.0,33.0,30.0,100.0,77.0,115.0,85.0,114.0,29.0


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [8]:
#Creating pandas dataframe

df_pums_2017 = pd.read_sql("""
    SELECT * 
    FROM pums_2017;
    """, conn)
df_pums_2017.shape 

(359075, 286)

In [9]:
df_pums_2017.head()

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000000006,9,01,11606,4,53,1061971,27.0,68.0,...,53.0,24.0,39.0,24.0,7.0,27.0,8.0,46.0,25.0,50.0
1,P,2013000000006,9,02,11606,4,53,1061971,22.0,66.0,...,49.0,21.0,38.0,20.0,7.0,25.0,8.0,41.0,22.0,47.0
2,P,2013000000012,9,01,10100,4,53,1061971,22.0,72.0,...,24.0,22.0,25.0,7.0,21.0,35.0,6.0,22.0,6.0,37.0
3,P,2013000000012,9,02,10100,4,53,1061971,19.0,64.0,...,21.0,18.0,19.0,7.0,17.0,29.0,6.0,19.0,6.0,29.0
4,P,2013000000038,9,01,11505,4,53,1061971,4.0,52.0,...,4.0,1.0,2.0,8.0,8.0,1.0,4.0,6.0,1.0,4.0


# Data Exploration

### Going through & understanding columns

#### sch = School enrollment
b .N/A (less than 3 years old)\
1 .No, has not attended in the last 3 months <-- this is what we're looking\
2 .Yes, public school or public college\
3 .Yes, private school or college or home school

In [20]:
df_pums_2017.sch.value_counts()

1    264532
2     69521
3     13390
Name: sch, dtype: int64

#### esr = Employment status recode

Looking for: 3 and 6

b .N/A (less than 16 years old)\
1 .Civilian employed, at work\
2 .Civilian employed, with a job but not at work\
3 .Unemployed <-- this is what we're interested in\
4 .Armed forces, at work\
5 .Armed forces, with a job but not at work\
6 .Not in labor force <-- this is what we're interested in

In [21]:
df_pums_2017.esr.value_counts()

1    160061
6    115658
3     10134
2      3513
4      2492
5         7
Name: esr, dtype: int64

#### puma = Public use microdata area code

53 11610	King County (Central)--Renton City, Fairwood, Bryn Mawr & Skyway PUMA\
53 11611	King County (West Central)--Burien, SeaTac, Tukwila Cities & White Center PUMA\
53 11612	King County (Far Southwest)--Federal Way, Des Moines Cities & Vashon Island PUMA\
53 11613	King County (Southwest Central)--Kent City PUMA\
53 11614	King County (Southwest)--Auburn City & Lakeland PUMA\
53 11615	King County (Southeast)--Maple Valley, Covington & Enumclaw Cities PUMA

In [22]:
df_pums_2017.puma.value_counts()

10200    18484
11505     9375
10100     9168
10600     8340
11801     8301
10400     8233
11300     7463
11608     7390
11705     7366
11000     7323
11609     7027
11706     7021
10902     6868
11104     6785
11602     6686
11401     6623
10800     6595
11607     6569
10502     6386
11601     6348
11605     6309
11200     6306
11802     6154
11501     6126
11101     6123
11615     6119
11611     6032
11606     5974
10501     5958
11900     5938
11704     5925
11614     5911
11506     5882
10503     5806
11610     5651
11504     5613
11502     5609
11616     5568
10703     5543
11102     5510
11402     5451
11507     5448
10504     5416
11613     5371
10300     5342
11103     5255
11701     5252
11703     5237
11503     5197
11603     5067
11612     5061
11604     5057
11702     5049
10701     5022
10901     4856
10702     4586
Name: puma, dtype: int64

#### schl = educational attainment

01 .No schooling completed\
02 .Nursery school, preschool\
03 .Kindergarten\
04 .Grade 1\
05 .Grade 2\
06 .Grade 3\
07 .Grade 4\
08 .Grade 5\
09 .Grade 6\
10 .Grade 7\
11 .Grade 8\
12 .Grade 9\
13 .Grade 10\
14 .Grade 11\
15 .12th grade - no diploma\
16 .Regular high school diploma\
17 .GED or alternative credential\
18 .Some college, but less than 1 year\
19 .1 or more years of college credit, no degree\
20 .Associate's degree\
21 .Bachelor's degree\
22 .Master's degree\
23 .Professional degree beyond a bachelor's degree\
24 .Doctorate degree

In [24]:
df_pums_2017.schl.value_counts()

21    57830
16    55002
19    48467
20    27153
18    24560
22    23770
17    11359
01    11138
14     8862
13     7590
12     6714
11     6479
09     5956
23     5728
15     5395
02     5181
10     4840
06     4815
08     4738
07     4596
05     4556
03     4501
04     4305
24     3908
Name: schl, dtype: int64

#### sex

1 .male\
2 .female

In [10]:
#filtering dataframe to include only columns we're interested in

df_pums_2017_filterd = df_pums_2017.filter(['puma', 'pwgtp', 'agep', 'nwav', 'nwlk', 'sch', 'schl',
                                  'sex', 'esr', 'pobp', 'rac1p', 'rac2p', 'rac3p', 'racaian', 'racasn',
                                  'racblk', 'racnh', 'racwht', 'waob'], axis = 1)

In [11]:
df_pums_2017_filterd2 = df_pums_2017_filterd.filter(['puma', 'pwgtp', 'agep', 'sch', 'schl', 'esr'])

In [13]:
df_pums_2017_filterd2.head()

,puma,pwgtp,agep,sch,schl,esr
0,11606,27.0,68.0,1,21,6
1,11606,22.0,66.0,1,22,6
2,10100,22.0,72.0,1,19,6
3,10100,19.0,64.0,1,21,6
4,11505,4.0,52.0,1,21,1


In [ ]:
df_pums_2017_filterd2.groupby()

In [13]:
df_pums_2017_filterd.head()

,puma,pwgtp,agep,nwav,nwlk,sch,schl,sex,esr,pobp,rac1p,rac2p,rac3p,racaian,racasn,racblk,racnh,racwht,waob
0,11606,27.0,68.0,5,2,1,21,1,6,012,1,01,001,0,0,0,0,1,1
1,11606,22.0,66.0,5,2,1,22,2,6,027,1,01,001,0,0,0,0,1,1
2,10100,22.0,72.0,5,2,1,19,1,6,301,1,01,001,0,0,0,0,1,7
3,10100,19.0,64.0,5,2,1,21,2,6,017,1,01,001,0,0,0,0,1,1
4,11505,4.0,52.0,5,3,1,21,1,1,053,1,01,001,0,0,0,0,1,1


## Data Preparation & Analysis
#### df_skc_oy_2017

In [83]:
#turning puma column into datatype 'int', so it can be filtered

df_pums_2017_filterd['puma']=df_pums_2017_filterd.puma.astype('int64')

In [66]:
esr = ['3', '6']
pumas_kc = [11606, 11607, 11608, 11609, 11610, 11611, 11612, 11613, 11614, 11615, 11616]
pumas_skc = [11610, 11611, 11612, 11613, 11614, 11615]
schl_no_dipl = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
schl_dipl_ged = ['16', '17']
schl_coll_nodeg = ['18','19']
schl_coll_deg = ['20', '21', '22', '23', '24']

In [22]:
#creating dataframe with oy conditions from sample dataset for only south king county

df_skc_oy_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(16, 24) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc) & 
                                  df_pums_2017_filterd.esr.isin(esr) & 
                                  (df_pums_2017_filterd.sch == '1')].sort_values('agep')

In [23]:
df_skc_oy_2017.head()

,puma,pwgtp,agep,nwav,nwlk,sch,schl,sex,esr,pobp,rac1p,rac2p,rac3p,racaian,racasn,racblk,racnh,racwht,waob
203547,11612,28.0,16.0,5,2,1,12,1,6,053,1,01,001,0,0,0,0,1,1
202778,11614,22.0,16.0,5,2,1,13,2,6,006,1,01,001,0,0,0,0,1,1
171097,11615,12.0,16.0,5,2,1,12,2,6,053,1,01,001,0,0,0,0,1,1
170532,11614,12.0,16.0,5,2,1,17,1,6,164,1,01,001,0,0,0,0,1,5
262050,11610,28.0,16.0,5,2,1,14,1,6,053,1,01,001,0,0,0,0,1,1


In [24]:
df_skc_oy_2017.shape

(391, 19)

In [99]:
#checking for NaNs

df_skc_oy_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 88069 to 24677
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   puma     391 non-null    int64  
 1   pwgtp    391 non-null    float64
 2   agep     391 non-null    float64
 3   nwav     391 non-null    object 
 4   nwlk     391 non-null    object 
 5   sch      391 non-null    object 
 6   schl     391 non-null    object 
 7   sex      391 non-null    object 
 8   esr      391 non-null    object 
 9   pobp     391 non-null    object 
 10  rac1p    391 non-null    object 
 11  rac2p    391 non-null    object 
 12  rac3p    391 non-null    object 
 13  racaian  391 non-null    object 
 14  racasn   391 non-null    object 
 15  racblk   391 non-null    object 
 16  racnh    391 non-null    object 
 17  racwht   391 non-null    object 
 18  waob     391 non-null    object 
dtypes: float64(2), int64(1), object(16)
memory usage: 61.1+ KB


In [27]:
#creating dataframe with oy conditions from sample dataset for all of king county

df_kc_oy_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(16, 24) & 
                                  df_pums_2017_filterd.puma.isin(pumas_kc) & 
                                  df_pums_2017_filterd.esr.isin(esr) & 
                                  (df_pums_2017_filterd.sch == '1')].sort_values('agep')

In [28]:
df_kc_oy_2017.shape

(594, 19)

In [29]:
#checking king county oy total by adding up pwgtp column (person weight to total population)

df_kc_oy_2017.pwgtp.sum()

15146.0

In [31]:
#checking south king county oy total by adding up pwgtp column (person weight to total population)

df_skc_oy_2017.pwgtp.sum()

10614.0

In [101]:
#oy breakdown by pumas within south king county

df_skc_oy_2017.groupby('puma')['pwgtp'].sum()

puma
11610    1853.0
11611    2038.0
11612    1977.0
11613    2006.0
11614    1530.0
11615    1210.0
Name: pwgtp, dtype: float64

In [102]:
#checking the increase in white race

df_skc_oy_2017.groupby('racwht')['pwgtp'].sum()

racwht
0    4427.0
1    6187.0
Name: pwgtp, dtype: float64

In [32]:
#checking sex ratio

df_skc_oy_2017.sex.value_counts(normalize=True) * 100

1    51.662404
2    48.337596
Name: sex, dtype: float64

#### Total 16 to 24 year population in south king county

In [138]:
df_skc_16to24_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(16, 24) & 
                                  df_pums_2017_filterd.puma.between(11610, 11615)].sort_values('agep')

In [139]:
df_skc_16to24_2017.pwgtp.sum()

85883.0

***
# PUMS_2016
### Compare methodology to see if we get similar numbers

In [80]:
df_oy_2016 = pd.read_csv('../../data/2016_pums/ss16pwa.csv')
df_oy_2016.head()

,RT,SERIALNO,SPORDER,PUMA,ST,ADJINC,PWGTP,AGEP,CIT,CITWP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2012000000014,1,10502,53,1056030,10,72,1,NaN,...,4,11,3,11,10,12,20,11,20,16
1,P,2012000000058,1,11802,53,1056030,35,58,1,NaN,...,38,35,35,35,35,36,11,58,35,10
2,P,2012000000066,1,10200,53,1056030,3,65,1,NaN,...,6,3,3,2,3,3,1,3,6,1
3,P,2012000000108,1,11501,53,1056030,34,47,1,NaN,...,8,36,59,37,48,56,10,34,13,35
4,P,2012000000247,1,11609,53,1056030,18,51,1,NaN,...,5,16,29,20,35,33,5,17,4,16


In [81]:
#checking dataset size between 2016 and 2017

print(df_oy_2016.shape, df_pums_2017.shape)

(353681, 283) (359075, 286)


In [79]:
#creating 2016 oy dataframe with our 2017 oy conditions (age, pumas, esr, sch)

df_oy_2016_app = df_oy_2016[df_oy_2016.AGEP.between(16, 24) & 
           df_oy_2016.PUMA.between(11610, 11615) & 
           df_oy_2016.ESR.isin(esr) & 
           (df_oy_2016.SCH == 1)].sort_values('AGEP')

In [41]:
df_oy_2016_app.head()

,RT,SERIALNO,SPORDER,PUMA,ST,ADJINC,PWGTP,AGEP,CIT,CITWP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
172472,P,2014000707906,3,11614,53,1022342,42,16,1,NaN,...,16,36,37,59,41,45,66,30,15,12
37820,P,2012000819734,4,11612,53,1056030,19,16,1,NaN,...,7,5,21,21,25,26,25,6,16,21
240397,P,2015000650843,3,11615,53,1013916,14,16,1,NaN,...,16,4,15,14,13,3,30,24,22,13
272079,P,2015001313106,4,11614,53,1013916,21,16,1,NaN,...,31,19,19,8,22,24,33,27,7,22
239832,P,2015000639047,5,11614,53,1013916,12,16,3,NaN,...,11,11,23,14,19,12,14,13,4,19


In [170]:
df_oy_2016_app.PWGTP.sum()

11705

***
# Breakdown by Age
### 16 to 18 OY Status by Age

In [54]:
#total 16 to 18 yr olds in skc

df_skc_16to18_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(16, 18) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc)].sort_values('agep')

In [55]:
df_skc_16to18_2017.pwgtp.sum()

30141.0

In [48]:
#total 16 to 18 yr old OY in skc

df_skc_oy_16to18_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(16, 18) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc) & 
                                  df_pums_2017_filterd.esr.isin(esr) & 
                                  (df_pums_2017_filterd.sch == '1')].sort_values('agep')

In [49]:
df_skc_oy_16to18_2017.pwgtp.sum()

1815.0

In [134]:
#16 to 18 yr old oy with no diploma in skc

df_skc_oy_16to18_2017[df_skc_oy_16to18_2017.schl.isin(schl_no_dipl)]['pwgtp'].sum()

916.0

In [50]:
#16 to 18 yr old oy with diploma or ged in skc

df_skc_oy_16to18_2017[df_skc_oy_16to18_2017.schl.isin(schl_dipl_ged)]['pwgtp'].sum()

781.0

In [51]:
#16 to 18 yr old oy with some college, but no degree, in skc

df_skc_oy_16to18_2017[df_skc_oy_16to18_2017.schl.isin(schl_coll_nodeg)]['pwgtp'].sum()

118.0

In [67]:
#16 to 18 yr old oy with degree in skc

df_skc_oy_16to18_2017[df_skc_oy_16to18_2017.schl.isin(schl_coll_deg)]['pwgtp'].sum()

0.0


### 19 to 21 OY Status by Age

In [56]:
#total 19 to 21 yr olds in skc

df_skc_19to21_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(19, 21) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc)].sort_values('agep')

In [57]:
df_skc_19to21_2017.pwgtp.sum()

25486.0

In [78]:
#total 19 to 21 yr old OY in skc

df_skc_oy_19to21_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(19, 21) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc) & 
                                  df_pums_2017_filterd.esr.isin(esr) & 
                                  (df_pums_2017_filterd.sch == '1')].sort_values('agep')

In [61]:
df_skc_oy_19to21_2017.pwgtp.sum()

3902.0

In [62]:
#19 to 21 yr old oy with no diploma in skc

df_skc_oy_19to21_2017[df_skc_oy_19to21_2017.schl.isin(schl_no_dipl)]['pwgtp'].sum()

1112.0

In [63]:
#19 to 21 yr old oy with diploma or ged in skc

df_skc_oy_19to21_2017[df_skc_oy_19to21_2017.schl.isin(schl_dipl_ged)]['pwgtp'].sum()

2176.0

In [64]:
#19 to 21 yr old oy with some college, but no degree, in skc

df_skc_oy_19to21_2017[df_skc_oy_19to21_2017.schl.isin(schl_coll_nodeg)]['pwgtp'].sum()

521.0

In [68]:
#19 to 21 yr old oy with degree in skc

df_skc_oy_19to21_2017[df_skc_oy_19to21_2017.schl.isin(schl_coll_deg)]['pwgtp'].sum()

93.0


### 22 to 24 OY Status by Age

In [69]:
#total 22 to 24 yr olds in skc

df_skc_22to24_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(22, 24) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc)].sort_values('agep')

In [70]:
df_skc_22to24_2017.pwgtp.sum()

30256.0

In [71]:
#total 22 to 24 yr old OY in skc

df_skc_oy_22to24_2017 = df_pums_2017_filterd[df_pums_2017_filterd.agep.between(22, 24) & 
                                  df_pums_2017_filterd.puma.isin(pumas_skc) & 
                                  df_pums_2017_filterd.esr.isin(esr) & 
                                  (df_pums_2017_filterd.sch == '1')].sort_values('agep')

In [72]:
df_skc_oy_22to24_2017.pwgtp.sum()

4897.0

In [73]:
#22 to 24 yr old oy with no diploma in skc

df_skc_oy_22to24_2017[df_skc_oy_22to24_2017.schl.isin(schl_no_dipl)]['pwgtp'].sum()

1349.0

In [74]:
#22 to 24 yr old oy with diploma or ged in skc

df_skc_oy_22to24_2017[df_skc_oy_22to24_2017.schl.isin(schl_dipl_ged)]['pwgtp'].sum()

2135.0

In [75]:
#22 to 24 yr old oy with some college, but no degree, in skc

df_skc_oy_22to24_2017[df_skc_oy_22to24_2017.schl.isin(schl_coll_nodeg)]['pwgtp'].sum()

1000.0

In [76]:
#22 to 24 yr old oy with degree in skc

df_skc_oy_22to24_2017[df_skc_oy_22to24_2017.schl.isin(schl_coll_deg)]['pwgtp'].sum()

413.0

Make sure you close the DB connection when you are done using it

### conn.close()